In [1]:
import pandas as pd

In [ ]:
def read_tbl():
    df = pd.read_excel("../data/input/Additional INC0090307.xlsx")

    df = df.rename(columns={"employeeid": "uid", 
                            "BirthDate": "birth_date", 
                            "AgencyName": "agency_name",
                            "StartDate":"start_date", 
                            "EndDate":"end_date",
                            "separationReason":"separation_reason",
                            "Status": "employment_status"})

    return df 


def split_name(fullname):
    parts = fullname.split(',')
    last_name = parts[0].strip()
    suffix = parts[1].strip() if len(parts) > 2 else ''
    first_middle = parts[-1].strip().split(' ')
    first_name = first_middle[0].strip()
    middle_name = ' '.join(first_middle[1:]).strip() if len(first_middle) > 1 else ''
    return first_name, middle_name, last_name, suffix


def clean_name(df):
     df.loc[:, "suffix"] = df.suffix.str.replace(r"\.", "", regex=True)
     df.loc[:, "middle_name"] = df.middle_name.str.replace(r"\.", "", regex=True)
     return df 


def strip_time_from_dates(df):
        df['start_date'] = df['start_date'].dt.date
        df['end_date'] = df['end_date'].dt.date
        return df


def extract_birth_year(df):
     df.loc[:, "year_of_birth"] = df.birth_date.astype(str).str.replace(r"^(\w+)-(.+)", r"\1", regex=True)
     return df 


df = read_tbl()

df[['first_name', 'middle_name', 'last_name', 'suffix']] = df['fullname'].apply(lambda x: pd.Series(split_name(x)))
df = df.pipe(strip_time_from_dates).pipe(extract_birth_year).pipe(clean_name)

df = df.drop(columns=["birth_date"])


df

,fullname,uid,agency_name,start_date,end_date,employment_status,separation_reason,first_name,middle_name,last_name,suffix,year_of_birth
0,"Aamodt, Jason R",133346,Reynoldsburg Police Department,2000-08-18,2022-05-20,Full-time,Retirement - Service retirement in good standi...,Jason,R,Aamodt,,1969
1,"Aaron, Jr., William Everett",213156,Goshen Twp. Police Department - Mahoning,2016-09-12,2017-03-30,Part-time,NaN,William,Everett,Aaron,Jr,1994
2,"Aaron, Jr., William Everett",213156,Mill Creek MetroParks Police Department,2019-11-22,2020-06-10,Reserve,NaN,William,Everett,Aaron,Jr,1994
3,"Aaron, Jr., William Everett",213156,Mill Creek MetroParks Police Department,2020-06-10,2021-09-30,Full-time,Resignation - In good standing (No Pending dis...,William,Everett,Aaron,Jr,1994
4,"Aaron, Jr., William Everett",213156,Mahoning County Sheriff's Office,2021-01-22,NaT,Reserve,NaN,William,Everett,Aaron,Jr,1994
...,...,...,...,...,...,...,...,...,...,...,...,...
152126,"Zwolinski, Zachary Paul",177560,North Randall Police Department,2013-03-25,2015-09-28,Auxiliary,NaN,Zachary,Paul,Zwolinski,,1987
152127,"Zwolinski, Zachary Paul",177560,Orange Village Police Department,2014-03-12,2015-09-09,Part-time,NaN,Zachary,Paul,Zwolinski,,1987
152128,"Zwolinski, Zachary Paul",177560,Orange Village Police Department,2015-09-09,NaT,Full-time,NaN,Zachary,Paul,Zwolinski,,1987
152129,"Zywczyk, Thomas L.",133925,Columbus Police Department,2001-01-05,2007-02-09,Full-time,NaN,Thomas,L,Zywczyk,,1976


In [3]:
df.to_csv("../data/output/ohio-processed.csv", index=False)